# Chapter 4 - Exercise 5: Photopolymerization Kinetics and Light-Based Bioprinting

## Learning Objectives
- Understand photopolymerization mechanisms in bioprinting
- Apply Beer-Lambert law to light penetration and absorption
- Model gelation kinetics and working curves
- Calculate exposure dose and cure depth relationships
- Optimize DLP/SLA parameters for resolution and cell viability
- Design multi-layer printing strategies

## Background

**Light-based bioprinting** uses photopolymerization to selectively solidify liquid photoresin containing cells. This enables:
- **High resolution** (10-100 µm)
- **Complex geometries** (overhangs, internal voids)
- **Fast layer production** (seconds per layer)

### Technologies (from Chapter 4.4.1):

**Stereolithography (SLA)**:
- Laser scans point-by-point to polymerize resin
- High precision but slower
- Typical wavelength: 365-405 nm (UV-Vis)

**Digital Light Processing (DLP)**:
- Projects entire layer pattern simultaneously
- Fast (1-10 s per layer)
- Pixel-limited resolution (25-100 µm)

**Two-Photon Polymerization**:
- Uses near-IR femtosecond laser
- Sub-micron resolution (<1 µm)
- Slow, expensive, but ultimate precision

### Key Physics:

**Beer-Lambert Law** (light attenuation):
$$I(z) = I_0 \cdot e^{-\alpha z}$$

where:
- I(z) = intensity at depth z
- I₀ = surface intensity
- α = absorption coefficient (mm⁻¹)
- z = penetration depth (mm)

**Working Curve** (cure depth vs exposure):
$$C_d = D_p \ln\left(\frac{E}{E_c}\right)$$

where:
- C_d = cure depth (µm)
- D_p = penetration depth = 1/α (µm)
- E = exposure dose (mJ/cm²)
- E_c = critical energy threshold (mJ/cm²)

**Exposure Dose**:
$$E = I \cdot t$$

- E = exposure dose (mJ/cm²)
- I = light intensity (mW/cm²)
- t = exposure time (s)

### Critical Parameters:
- **Wavelength**: 365-405 nm (UV-A, safer for cells)
- **Photoinitiator**: LAP, Irgacure 2959 (cell-compatible)
- **Concentration**: 0.05-0.5% (w/v) typical
- **Exposure**: 10-1000 mJ/cm² (cell-dependent)
- **Layer thickness**: 25-100 µm

## Setup: Install and Import Libraries

In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install numpy matplotlib pandas seaborn scipy plotly -q

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.optimize import curve_fit, fsolve
from scipy.integrate import odeint
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✓ All libraries loaded successfully!")
print("Ready to explore photopolymerization kinetics!")

## Part 1: Beer-Lambert Law and Light Penetration

In [ ]:
def calculate_light_intensity(z, I0, alpha):
    """
    Calculate light intensity at depth using Beer-Lambert law.
    
    Parameters:
    -----------
    z : float or array
        Depth into material (µm)
    I0 : float
        Surface intensity (mW/cm²)
    alpha : float
        Absorption coefficient (mm⁻¹)
    
    Returns:
    --------
    I : float or array
        Intensity at depth z (mW/cm²)
    """
    z_mm = z / 1000  # Convert µm to mm
    I = I0 * np.exp(-alpha * z_mm)
    return I

def calculate_cure_depth(E, Ec, Dp):
    """
    Calculate cure depth using working curve equation.
    
    Parameters:
    -----------
    E : float or array
        Exposure dose (mJ/cm²)
    Ec : float
        Critical energy threshold (mJ/cm²)
    Dp : float
        Penetration depth (µm)
    
    Returns:
    --------
    Cd : float or array
        Cure depth (µm)
    """
    # Only cure where E > Ec
    Cd = np.zeros_like(E)
    mask = E > Ec
    Cd[mask] = Dp * np.log(E[mask] / Ec)
    return Cd

def calculate_exposure_dose(intensity, time):
    """
    Calculate exposure dose.
    
    Parameters:
    -----------
    intensity : float
        Light intensity (mW/cm²)
    time : float
        Exposure time (s)
    
    Returns:
    --------
    E : float
        Exposure dose (mJ/cm²)
    """
    E = intensity * time
    return E

def calculate_absorption_coefficient(photoinitiator_conc, extinction_coeff, additional_absorbers=0):
    """
    Calculate absorption coefficient from photoinitiator properties.
    
    Parameters:
    -----------
    photoinitiator_conc : float
        Photoinitiator concentration (% w/v)
    extinction_coeff : float
        Molar extinction coefficient at wavelength (L/(mol·cm))
    additional_absorbers : float
        Additional absorption from bioink components (mm⁻¹)
    
    Returns:
    --------
    alpha : float
        Absorption coefficient (mm⁻¹)
    """
    # Assume typical photoinitiator MW ~200 g/mol
    MW = 200  # g/mol
    
    # Convert % w/v to mol/L
    # 0.1% w/v = 1 g/L
    conc_gL = photoinitiator_conc * 10  # g/L
    conc_molL = conc_gL / MW  # mol/L
    
    # Beer's law: A = ε·c·l → α = 2.303·ε·c (for decadic absorption)
    alpha_PI = 2.303 * extinction_coeff * conc_molL / 10  # Convert to mm⁻¹
    
    alpha_total = alpha_PI + additional_absorbers
    
    return alpha_total

def predict_cell_viability_light(exposure_dose, wavelength=405):
    """
    Predict cell viability based on light exposure.
    Based on empirical relationships from literature.
    
    Parameters:
    -----------
    exposure_dose : float or array
        Total exposure dose (mJ/cm²)
    wavelength : float
        Wavelength (nm): 365, 385, or 405
    
    Returns:
    --------
    viability : float or array
        Predicted cell viability (%)
    """
    # Wavelength-dependent toxicity
    if wavelength <= 365:
        # UV-A: more toxic
        E_50 = 500  # mJ/cm² at 50% viability
        slope = 0.005
    elif wavelength <= 385:
        # Near-UV
        E_50 = 800
        slope = 0.003
    else:
        # Visible (405 nm): least toxic
        E_50 = 1200
        slope = 0.002
    
    # Sigmoidal decay
    viability = 95 / (1 + np.exp(slope * (exposure_dose - E_50)))
    viability = np.clip(viability, 0, 100)
    
    return viability

print("✓ Photopolymerization calculation functions defined!")
print("\nKey functions:")
print("  • calculate_light_intensity() - Beer-Lambert law")
print("  • calculate_cure_depth() - Working curve")
print("  • calculate_exposure_dose()")
print("  • calculate_absorption_coefficient()")
print("  • predict_cell_viability_light()")

## Part 2: Visualize Beer-Lambert Law and Light Penetration

In [ ]:
# Create depth range
depth_range = np.linspace(0, 500, 500)  # µm

# Different photoinitiator concentrations
I0_fixed = 10  # mW/cm²
PI_concentrations = [0.05, 0.1, 0.2, 0.5]  # % w/v
extinction_coeff = 100  # L/(mol·cm) - typical for LAP at 405 nm

colors_conc = ['#3498db', '#2ecc71', '#f39c12', '#e74c3c']

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Intensity profiles for different concentrations
for conc, color in zip(PI_concentrations, colors_conc):
    alpha = calculate_absorption_coefficient(conc, extinction_coeff)
    intensity_profile = calculate_light_intensity(depth_range, I0_fixed, alpha)
    
    ax1.plot(depth_range, intensity_profile, linewidth=3, color=color,
            label=f'{conc}% PI (α={alpha:.2f} mm⁻¹)')
    
    # Mark penetration depth (1/e intensity)
    Dp = 1000 / alpha  # µm
    I_at_Dp = I0_fixed / np.e
    ax1.plot(Dp, I_at_Dp, 'o', markersize=10, color=color, 
            markeredgecolor='black', markeredgewidth=2)

ax1.axhline(I0_fixed * 0.1, color='red', linestyle='--', linewidth=2,
           label='10% intensity threshold')
ax1.set_xlabel('Depth (µm)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Intensity (mW/cm²)', fontsize=12, fontweight='bold')
ax1.set_title('Light Penetration (Beer-Lambert Law)\nI(z) = I₀·exp(-αz)',
             fontsize=13, fontweight='bold')
ax1.legend(loc='best', frameon=True, fancybox=True)
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, I0_fixed * 1.1)

# Plot 2: Penetration depth vs concentration
conc_range = np.linspace(0.01, 0.5, 100)
Dp_values = []

for conc in conc_range:
    alpha = calculate_absorption_coefficient(conc, extinction_coeff)
    Dp = 1000 / alpha  # µm
    Dp_values.append(Dp)

ax2.plot(conc_range, Dp_values, linewidth=3, color='#9b59b6')
ax2.axhspan(50, 150, alpha=0.2, color='green', 
           label='Target layer thickness range')

# Mark typical concentrations
for conc in PI_concentrations:
    alpha = calculate_absorption_coefficient(conc, extinction_coeff)
    Dp = 1000 / alpha
    ax2.plot(conc, Dp, 'o', markersize=12, color='red',
            markeredgecolor='black', markeredgewidth=2)
    ax2.annotate(f'{conc}%', xy=(conc, Dp), xytext=(10, 10),
                textcoords='offset points', fontsize=10, fontweight='bold')

ax2.set_xlabel('Photoinitiator Concentration (% w/v)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Penetration Depth, Dₚ (µm)', fontsize=12, fontweight='bold')
ax2.set_title('Penetration Depth vs Concentration\nDₚ = 1/α',
             fontsize=13, fontweight='bold')
ax2.legend(loc='best', frameon=True, fancybox=True)
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 500)

# Plot 3: Normalized intensity (logarithmic view)
for conc, color in zip(PI_concentrations, colors_conc):
    alpha = calculate_absorption_coefficient(conc, extinction_coeff)
    intensity_profile = calculate_light_intensity(depth_range, I0_fixed, alpha)
    intensity_normalized = intensity_profile / I0_fixed
    
    ax3.semilogy(depth_range, intensity_normalized, linewidth=3, color=color,
                label=f'{conc}% PI')

ax3.axhline(np.exp(-1), color='black', linestyle='--', linewidth=2,
           label='1/e ≈ 0.37 (Dₚ)')
ax3.axhline(0.1, color='red', linestyle='--', linewidth=2,
           label='10% intensity')

ax3.set_xlabel('Depth (µm)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Normalized Intensity (I/I₀)', fontsize=12, fontweight='bold')
ax3.set_title('Light Attenuation (Log Scale)\nShowing Exponential Decay',
             fontsize=13, fontweight='bold')
ax3.legend(loc='best', frameon=True, fancybox=True, fontsize=9)
ax3.grid(True, alpha=0.3, which='both')
ax3.set_ylim(0.01, 1.2)

# Plot 4: Exposure dose profiles at different exposure times
exposure_times = [1, 2, 5, 10]  # seconds
conc_selected = 0.1  # % w/v
alpha_selected = calculate_absorption_coefficient(conc_selected, extinction_coeff)

for t, color in zip(exposure_times, colors_conc):
    intensity_profile = calculate_light_intensity(depth_range, I0_fixed, alpha_selected)
    exposure_profile = intensity_profile * t  # mJ/cm²
    
    ax4.plot(depth_range, exposure_profile, linewidth=3, color=color,
            label=f't = {t} s')

# Add typical critical energy thresholds
Ec_values = [10, 20, 50]  # mJ/cm²
for Ec in Ec_values:
    ax4.axhline(Ec, color='gray', linestyle=':', linewidth=1.5, alpha=0.7)
    ax4.text(450, Ec + 2, f'Eₓ={Ec}', fontsize=9)

ax4.set_xlabel('Depth (µm)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Exposure Dose (mJ/cm²)', fontsize=12, fontweight='bold')
ax4.set_title(f'Exposure Dose Profiles\n({conc_selected}% PI, I₀={I0_fixed} mW/cm²)',
             fontsize=13, fontweight='bold')
ax4.legend(loc='best', frameon=True, fancybox=True)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Key Observations:")
print("   • Higher PI concentration → faster light attenuation")
print("   • Penetration depth Dₚ = 1/α decreases with concentration")
print("   • Intensity decays exponentially with depth (Beer-Lambert)")
print("   • At depth Dₚ, intensity drops to 37% (1/e) of surface value")
print("   • Exposure dose = Intensity × Time")
print("   • Must balance: high PI (fast cure) vs deep penetration (thick layers)")

## Part 3: Working Curves and Cure Depth Analysis

The working curve relates exposure dose to cure depth:

In [ ]:
# Parameters for different photoresins
photoresins = {
    'GelMA (0.1% LAP)': {'Ec': 15, 'Dp': 200, 'color': '#3498db'},
    'PEGDA (0.2% I2959)': {'Ec': 25, 'Dp': 150, 'color': '#2ecc71'},
    'HA-MA (0.05% LAP)': {'Ec': 10, 'Dp': 250, 'color': '#f39c12'},
    'High PI (0.5%)': {'Ec': 30, 'Dp': 80, 'color': '#e74c3c'}
}

# Exposure range
exposure_range = np.linspace(5, 200, 500)  # mJ/cm²

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Working curves for different resins
for resin_name, props in photoresins.items():
    cure_depth = calculate_cure_depth(exposure_range, props['Ec'], props['Dp'])
    ax1.plot(exposure_range, cure_depth, linewidth=3, color=props['color'],
            label=resin_name)
    
    # Mark critical energy
    ax1.plot(props['Ec'], 0, 'v', markersize=12, color=props['color'],
            markeredgecolor='black', markeredgewidth=2)

# Add target layer thickness range
ax1.axhspan(50, 100, alpha=0.2, color='green', label='Target layer thickness')

ax1.set_xlabel('Exposure Dose, E (mJ/cm²)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Cure Depth, Cₐ (µm)', fontsize=12, fontweight='bold')
ax1.set_title('Working Curves\nCₐ = Dₚ·ln(E/Eₓ)',
             fontsize=13, fontweight='bold')
ax1.legend(loc='best', frameon=True, fancybox=True, fontsize=9)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 200)
ax1.set_ylim(0, 350)

# Plot 2: Effect of Dp (penetration depth)
Ec_fixed = 20  # mJ/cm²
Dp_values = [50, 100, 200, 300]  # µm
colors_Dp = ['#8e44ad', '#3498db', '#2ecc71', '#f39c12']

for Dp, color in zip(Dp_values, colors_Dp):
    cure_depth = calculate_cure_depth(exposure_range, Ec_fixed, Dp)
    ax2.plot(exposure_range, cure_depth, linewidth=3, color=color,
            label=f'Dₚ = {Dp} µm')

ax2.axhspan(50, 100, alpha=0.2, color='green')
ax2.set_xlabel('Exposure Dose, E (mJ/cm²)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Cure Depth, Cₐ (µm)', fontsize=12, fontweight='bold')
ax2.set_title(f'Effect of Penetration Depth\n(Eₓ = {Ec_fixed} mJ/cm² fixed)',
             fontsize=13, fontweight='bold')
ax2.legend(loc='best', frameon=True, fancybox=True)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 200)
ax2.set_ylim(0, 400)

# Plot 3: Effect of Ec (critical energy)
Dp_fixed = 150  # µm
Ec_values = [5, 10, 20, 40]  # mJ/cm²
colors_Ec = ['#e74c3c', '#f39c12', '#2ecc71', '#3498db']

for Ec, color in zip(Ec_values, colors_Ec):
    cure_depth = calculate_cure_depth(exposure_range, Ec, Dp_fixed)
    ax3.plot(exposure_range, cure_depth, linewidth=3, color=color,
            label=f'Eₓ = {Ec} mJ/cm²')

ax3.axhspan(50, 100, alpha=0.2, color='green')
ax3.set_xlabel('Exposure Dose, E (mJ/cm²)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Cure Depth, Cₐ (µm)', fontsize=12, fontweight='bold')
ax3.set_title(f'Effect of Critical Energy\n(Dₚ = {Dp_fixed} µm fixed)',
             fontsize=13, fontweight='bold')
ax3.legend(loc='best', frameon=True, fancybox=True)
ax3.grid(True, alpha=0.3)
ax3.set_xlim(0, 200)
ax3.set_ylim(0, 300)

# Plot 4: Required exposure for target layer thickness
target_thicknesses = np.linspace(25, 150, 6)  # µm
Dp_range = np.linspace(50, 300, 100)  # µm
Ec_test = 20  # mJ/cm²

for target_thickness in target_thicknesses:
    # From Cd = Dp·ln(E/Ec), solve for E:
    # E = Ec·exp(Cd/Dp)
    required_exposure = Ec_test * np.exp(target_thickness / Dp_range)
    
    ax4.plot(Dp_range, required_exposure, linewidth=2.5,
            label=f'Layer = {target_thickness:.0f} µm')

# Add practical exposure range
ax4.axhspan(10, 100, alpha=0.2, color='green', label='Practical exposure range')

ax4.set_xlabel('Penetration Depth, Dₚ (µm)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Required Exposure, E (mJ/cm²)', fontsize=12, fontweight='bold')
ax4.set_title(f'Exposure Requirements\n(Eₓ = {Ec_test} mJ/cm²)',
             fontsize=13, fontweight='bold')
ax4.legend(loc='best', frameon=True, fancybox=True, fontsize=9)
ax4.grid(True, alpha=0.3)
ax4.set_ylim(0, 200)

plt.tight_layout()
plt.show()

print("\n💡 Working Curve Insights:")
print("   • Cure depth increases logarithmically with exposure")
print("   • Higher Dₚ (lower PI conc) → deeper cure per dose")
print("   • Lower Eₓ (more reactive) → cures at lower doses")
print("   • For 50-100 µm layers: need E = 20-80 mJ/cm² typically")
print("   • Must calibrate working curve for each resin formulation")
print("   • Trade-off: Fast cure (high PI) vs thick layers (low PI)")

## Part 4: Interactive DLP/SLA Parameter Optimizer

### 🎯 STUDENT TASK 1: Design a Light-Based Bioprinting Process

**Optimize parameters for your target geometry and cell type:**

In [ ]:
# ==========================================
# STUDENT PARAMETERS - MODIFY THESE VALUES
# ==========================================

# Photoresin properties
PI_concentration = 0.1        # % w/v (try: 0.05, 0.1, 0.2, 0.5)
extinction_coefficient = 100   # L/(mol·cm) - typical for LAP/I2959
critical_energy = 20          # mJ/cm² (try: 10, 20, 30, 40)

# Light source
wavelength = 405              # nm (365, 385, or 405)
light_intensity = 10          # mW/cm² (try: 5, 10, 20, 40)

# Print parameters
target_layer_thickness = 75   # µm (try: 50, 75, 100)
total_height = 2000           # µm (2 mm construct)

# ==========================================

# Calculate derived parameters
alpha = calculate_absorption_coefficient(PI_concentration, extinction_coefficient)
Dp = 1000 / alpha  # µm

# Calculate required exposure time for target layer thickness
# From Cd = Dp·ln(E/Ec) and E = I·t:
# Cd = Dp·ln(I·t/Ec)
# t = (Ec/I)·exp(Cd/Dp)
required_exposure_dose = critical_energy * np.exp(target_layer_thickness / Dp)
required_exposure_time = required_exposure_dose / light_intensity

# Calculate actual cure depth with this exposure
actual_cure_depth = calculate_cure_depth(required_exposure_dose, critical_energy, Dp)

# Calculate number of layers
num_layers = int(np.ceil(total_height / target_layer_thickness))

# Calculate total print time (layer time + recoating time ~5s)
recoat_time = 5  # seconds
total_time_per_layer = required_exposure_time + recoat_time
total_print_time = num_layers * total_time_per_layer

# Calculate cell viability
cell_viability = predict_cell_viability_light(required_exposure_dose, wavelength)

# Calculate Z-resolution (vertical)
# Assuming 90-10% cure threshold
E_90 = critical_energy * np.exp(target_layer_thickness / Dp)
E_10 = E_90 * 0.9
z_resolution = Dp * np.log(E_90 / E_10)

# Create comprehensive visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Cure Profile', 'Cell Viability',
                   'Layer-by-Layer Build', 'Process Summary'),
    specs=[[{'type': 'scatter'}, {'type': 'indicator'}],
           [{'type': 'scatter'}, {'type': 'table'}]]
)

# Plot 1: Cure profile
depth_profile = np.linspace(0, 200, 200)
intensity_profile = calculate_light_intensity(depth_profile, light_intensity, alpha)
exposure_profile = intensity_profile * required_exposure_time

fig.add_trace(
    go.Scatter(x=depth_profile, y=exposure_profile,
              mode='lines', name='Exposure dose',
              line=dict(color='blue', width=3)),
    row=1, col=1
)

# Add critical energy threshold
fig.add_hline(y=critical_energy, line_dash="dash", line_color="red",
             annotation_text=f"Eₓ = {critical_energy} mJ/cm²",
             row=1, col=1)

# Add cure depth marker
fig.add_vline(x=actual_cure_depth, line_dash="dot", line_color="green",
             annotation_text=f"Cure depth = {actual_cure_depth:.0f} µm",
             row=1, col=1)

fig.update_xaxes(title_text="Depth (µm)", row=1, col=1)
fig.update_yaxes(title_text="Exposure Dose (mJ/cm²)", row=1, col=1)

# Plot 2: Cell viability gauge
if cell_viability >= 85:
    gauge_color = "green"
    status = "Excellent"
elif cell_viability >= 70:
    gauge_color = "yellow"
    status = "Acceptable"
else:
    gauge_color = "red"
    status = "Poor"

fig.add_trace(
    go.Indicator(
        mode="gauge+number+delta",
        value=cell_viability,
        title={'text': f"Cell Viability<br>{status}"},
        delta={'reference': 85},
        gauge={
            'axis': {'range': [None, 100]},
            'bar': {'color': gauge_color},
            'steps': [
                {'range': [0, 70], 'color': "lightgray"},
                {'range': [70, 85], 'color': "lightyellow"},
                {'range': [85, 100], 'color': "lightgreen"}],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 85}
        }),
    row=1, col=2
)

# Plot 3: Layer-by-layer schematic
# Show first 10 layers
layers_to_show = min(10, num_layers)
for i in range(layers_to_show):
    y_bottom = i * target_layer_thickness
    y_top = (i + 1) * target_layer_thickness
    
    fig.add_shape(
        type="rect",
        x0=0, x1=500, y0=y_bottom, y1=y_top,
        fillcolor='lightblue' if i % 2 == 0 else 'lightgreen',
        line=dict(color='black', width=1),
        row=2, col=1
    )
    
    fig.add_annotation(
        x=250, y=(y_bottom + y_top) / 2,
        text=f"Layer {i+1}",
        showarrow=False,
        font=dict(size=10),
        row=2, col=1
    )

fig.update_xaxes(title_text="Width (µm)", row=2, col=1)
fig.update_yaxes(title_text="Height (µm)", row=2, col=1)

# Plot 4: Process summary table
fig.add_trace(
    go.Table(
        header=dict(values=['Parameter', 'Value'],
                   fill_color='paleturquoise',
                   align='left',
                   font=dict(size=12)),
        cells=dict(values=[
            ['Absorption Coeff (α)', 'Penetration Depth (Dₚ)', 'Critical Energy (Eₓ)',
             'Exposure Time', 'Cure Depth', 'Number of Layers',
             'Total Print Time', 'Cell Viability', 'Z-Resolution'],
            [f'{alpha:.2f} mm⁻¹', f'{Dp:.0f} µm', f'{critical_energy} mJ/cm²',
             f'{required_exposure_time:.1f} s', f'{actual_cure_depth:.0f} µm',
             f'{num_layers}', f'{total_print_time/60:.1f} min',
             f'{cell_viability:.1f}%', f'{z_resolution:.0f} µm']
        ],
        fill_color='lavender',
        align='left',
        font=dict(size=11))),
    row=2, col=2
)

fig.update_layout(height=900, showlegend=False,
                 title_text=f"DLP/SLA Process Design - {wavelength} nm, {PI_concentration}% PI")
fig.show()

# Detailed text output
print("="*70)
print("LIGHT-BASED BIOPRINTING PROCESS DESIGN")
print("="*70)
print(f"\nPhotoresin Properties:")
print(f"  • Photoinitiator concentration: {PI_concentration}% w/v")
print(f"  • Critical energy (Eₓ): {critical_energy} mJ/cm²")
print(f"  • Absorption coefficient (α): {alpha:.2f} mm⁻¹")
print(f"  • Penetration depth (Dₚ): {Dp:.0f} µm")

print(f"\nLight Source:")
print(f"  • Wavelength: {wavelength} nm")
print(f"  • Intensity: {light_intensity} mW/cm²")

print(f"\nPrint Parameters:")
print(f"  • Target layer thickness: {target_layer_thickness} µm")
print(f"  • Required exposure: {required_exposure_dose:.1f} mJ/cm²")
print(f"  • Exposure time: {required_exposure_time:.1f} s")
print(f"  • Actual cure depth: {actual_cure_depth:.0f} µm")

print(f"\nConstruct Specifications:")
print(f"  • Total height: {total_height} µm ({total_height/1000:.1f} mm)")
print(f"  • Number of layers: {num_layers}")
print(f"  • Time per layer: {total_time_per_layer:.1f} s")
print(f"  • Total print time: {total_print_time/60:.1f} min")
print(f"  • Z-resolution: {z_resolution:.0f} µm")

print(f"\nCell Viability:")
print(f"  • Predicted viability: {cell_viability:.1f}%")
print(f"  • Exposure dose: {required_exposure_dose:.1f} mJ/cm²")

print(f"\nAssessment:")
if abs(actual_cure_depth - target_layer_thickness) / target_layer_thickness < 0.1:
    print("  ✅ Cure depth matches target well (<10% error)")
elif actual_cure_depth > target_layer_thickness * 1.5:
    print("  ⚠️  Significant overcure - may lose Z-resolution")
    print("     → Consider: reducing exposure time or increasing PI concentration")
else:
    print("  ⚠️  Undercure - layers may not bond properly")
    print("     → Consider: increasing exposure time or reducing PI concentration")

if cell_viability >= 85:
    print("  ✅ Excellent cell viability expected")
elif cell_viability >= 70:
    print("  ✓ Acceptable cell viability")
else:
    print("  ❌ Poor cell viability - excessive light exposure")
    print("     → Consider: reducing intensity, shorter time, or longer wavelength")

if required_exposure_time < 1:
    print("  ⚠️  Very short exposure time - may need higher accuracy equipment")
elif required_exposure_time > 30:
    print("  ⚠️  Long exposure time - slow printing")
    print("     → Consider: increasing light intensity")
else:
    print("  ✓ Exposure time is practical")

if total_print_time / 60 > 60:
    print("  ⚠️  Print time exceeds 1 hour - consider thicker layers or higher intensity")
else:
    print("  ✓ Print time is reasonable")

print("\n" + "="*70)

## Part 5: Design Space Exploration - Multi-Parameter Optimization

In [ ]:
# Create parameter sweep
PI_concs = np.linspace(0.05, 0.5, 50)
intensities = np.linspace(5, 40, 50)

PI_grid, I_grid = np.meshgrid(PI_concs, intensities)

# Fixed parameters
extinction_fixed = 100
Ec_fixed = 20
target_thickness = 75  # µm
wavelength_fixed = 405

# Calculate metrics for each combination
exposure_time_grid = np.zeros_like(PI_grid)
viability_grid = np.zeros_like(PI_grid)
cure_depth_grid = np.zeros_like(PI_grid)

for i in range(PI_grid.shape[0]):
    for j in range(PI_grid.shape[1]):
        PI_conc = PI_grid[i, j]
        intensity = I_grid[i, j]
        
        # Calculate alpha and Dp
        alpha = calculate_absorption_coefficient(PI_conc, extinction_fixed)
        Dp = 1000 / alpha
        
        # Calculate required exposure for target thickness
        E_required = Ec_fixed * np.exp(target_thickness / Dp)
        t_required = E_required / intensity
        
        # Calculate viability
        viability = predict_cell_viability_light(E_required, wavelength_fixed)
        
        # Calculate actual cure depth
        cure_depth = calculate_cure_depth(E_required, Ec_fixed, Dp)
        
        exposure_time_grid[i, j] = t_required
        viability_grid[i, j] = viability
        cure_depth_grid[i, j] = cure_depth

# Create visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 14))

# Plot 1: Exposure time
contour1 = ax1.contourf(PI_grid, I_grid, exposure_time_grid, levels=20, cmap='viridis')
contour1_lines = ax1.contour(PI_grid, I_grid, exposure_time_grid, 
                             levels=[2, 5, 10, 20], colors='white', linewidths=2)
ax1.clabel(contour1_lines, inline=True, fontsize=10, fmt='%d s')
cbar1 = plt.colorbar(contour1, ax=ax1)
cbar1.set_label('Exposure Time (s)', fontsize=11, fontweight='bold')

ax1.set_xlabel('Photoinitiator Concentration (% w/v)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Light Intensity (mW/cm²)', fontsize=12, fontweight='bold')
ax1.set_title(f'Exposure Time for {target_thickness} µm Layer',
             fontsize=13, fontweight='bold')

# Plot 2: Cell viability
contour2 = ax2.contourf(PI_grid, I_grid, viability_grid, levels=20, cmap='RdYlGn')
contour2_lines = ax2.contour(PI_grid, I_grid, viability_grid,
                             levels=[70, 80, 85, 90], colors='black', linewidths=2)
ax2.clabel(contour2_lines, inline=True, fontsize=10, fmt='%d%%')
cbar2 = plt.colorbar(contour2, ax=ax2)
cbar2.set_label('Cell Viability (%)', fontsize=11, fontweight='bold')

ax2.set_xlabel('Photoinitiator Concentration (% w/v)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Light Intensity (mW/cm²)', fontsize=12, fontweight='bold')
ax2.set_title(f'Predicted Cell Viability ({wavelength_fixed} nm)',
             fontsize=13, fontweight='bold')

# Plot 3: Cure depth accuracy
cure_depth_error = np.abs(cure_depth_grid - target_thickness) / target_thickness * 100
contour3 = ax3.contourf(PI_grid, I_grid, cure_depth_error, levels=20, cmap='RdYlGn_r')
contour3_lines = ax3.contour(PI_grid, I_grid, cure_depth_error,
                             levels=[5, 10, 20], colors='black', linewidths=2)
ax3.clabel(contour3_lines, inline=True, fontsize=10, fmt='%d%%')
cbar3 = plt.colorbar(contour3, ax=ax3)
cbar3.set_label('Cure Depth Error (%)', fontsize=11, fontweight='bold')

ax3.set_xlabel('Photoinitiator Concentration (% w/v)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Light Intensity (mW/cm²)', fontsize=12, fontweight='bold')
ax3.set_title('Layer Thickness Accuracy',
             fontsize=13, fontweight='bold')

# Plot 4: Optimal operating zone
# Define optimal: viability >85%, exposure 2-15s, cure error <10%
optimal_zone = (viability_grid >= 85) & \
               (exposure_time_grid >= 2) & (exposure_time_grid <= 15) & \
               (cure_depth_error < 10)

acceptable_zone = (viability_grid >= 75) & \
                 (exposure_time_grid >= 1) & (exposure_time_grid <= 30) & \
                 (cure_depth_error < 20)

ax4.contourf(PI_grid, I_grid, optimal_zone.astype(float),
            levels=[0.5, 1.5], colors=['white', 'lightgreen'], alpha=0.7)
ax4.contourf(PI_grid, I_grid, acceptable_zone.astype(float),
            levels=[0.5, 1.5], colors=['white', 'lightyellow'], alpha=0.5)

# Add boundary lines
ax4.contour(PI_grid, I_grid, viability_grid, levels=[85],
           colors=['green'], linewidths=3, linestyles='solid')
ax4.contour(PI_grid, I_grid, exposure_time_grid, levels=[2, 15],
           colors=['blue'], linewidths=2, linestyles='dashed')

ax4.set_xlabel('Photoinitiator Concentration (% w/v)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Light Intensity (mW/cm²)', fontsize=12, fontweight='bold')
ax4.set_title('Optimal Operating Zone\n(Green: Optimal, Yellow: Acceptable)',
             fontsize=13, fontweight='bold')

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='lightgreen', edgecolor='green', linewidth=2,
          label='Optimal (Viab≥85%, 2-15s, <10% error)'),
    Patch(facecolor='lightyellow', edgecolor='orange', linewidth=2,
          label='Acceptable (Viab≥75%, 1-30s, <20% error)')
]
ax4.legend(handles=legend_elements, loc='upper right', frameon=True, fancybox=True)

plt.tight_layout()
plt.show()

print("\n💡 Design Space Analysis:")
print("   • Lower PI concentration → longer exposure time needed")
print("   • Higher intensity → shorter exposure time")
print("   • Sweet spot: 0.1-0.2% PI with 10-20 mW/cm²")
print("   • High intensity + high PI → fast but poor viability")
print("   • Low intensity + low PI → good viability but very slow")
print("   • Optimal zone balances speed, accuracy, and cell survival")

## Part 6: Wavelength Comparison and Cell Safety

In [ ]:
# Compare different wavelengths
wavelengths = [365, 385, 405]  # nm
wavelength_names = ['365 nm (UV-A)', '385 nm (Near-UV)', '405 nm (Visible)']
colors_wavelength = ['#8e44ad', '#3498db', '#2ecc71']

exposure_range_wl = np.linspace(0, 2000, 500)  # mJ/cm²

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Viability vs exposure for different wavelengths
for wl, wl_name, color in zip(wavelengths, wavelength_names, colors_wavelength):
    viability_curve = predict_cell_viability_light(exposure_range_wl, wl)
    ax1.plot(exposure_range_wl, viability_curve, linewidth=3, color=color,
            label=wl_name)

# Add practical exposure range
ax1.axvspan(20, 200, alpha=0.2, color='green', label='Typical exposure range')
ax1.axhline(85, color='red', linestyle='--', linewidth=2, label='Target viability')

ax1.set_xlabel('Exposure Dose (mJ/cm²)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Cell Viability (%)', fontsize=12, fontweight='bold')
ax1.set_title('Wavelength-Dependent Photodamage',
             fontsize=13, fontweight='bold')
ax1.legend(loc='best', frameon=True, fancybox=True)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 2000)
ax1.set_ylim(0, 100)

# Plot 2: Safe exposure window for each wavelength
# Calculate max safe exposure (for 85% viability)
safe_exposures = []
for wl in wavelengths:
    for E in exposure_range_wl:
        viab = predict_cell_viability_light(E, wl)
        if viab < 85:
            safe_exposures.append(E)
            break
    else:
        safe_exposures.append(exposure_range_wl[-1])

bars = ax2.barh(wavelength_names, safe_exposures, color=colors_wavelength,
               alpha=0.7, edgecolor='black', linewidth=2)

for bar, value in zip(bars, safe_exposures):
    ax2.text(value + 50, bar.get_y() + bar.get_height()/2,
            f'{value:.0f} mJ/cm²',
            va='center', fontweight='bold', fontsize=11)

ax2.set_xlabel('Maximum Safe Exposure (mJ/cm²)', fontsize=12, fontweight='bold')
ax2.set_title('Safe Exposure Window\n(For ≥85% Viability)',
             fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

# Plot 3: Photoinitiator absorption spectra (qualitative)
# Typical absorption profiles for common PIs
wavelength_spectrum = np.linspace(300, 450, 200)

# LAP: peaks around 365-385 nm
LAP_absorption = 1.0 * np.exp(-((wavelength_spectrum - 375)**2) / (2 * 20**2))
# Irgacure 2959: peaks around 365 nm  
I2959_absorption = 0.8 * np.exp(-((wavelength_spectrum - 365)**2) / (2 * 15**2))
# Eosin Y: peaks around 520 nm (shifted for visibility)
EosinY_absorption = 0.6 * np.exp(-((wavelength_spectrum - 420)**2) / (2 * 40**2))

ax3.plot(wavelength_spectrum, LAP_absorption, linewidth=3,
        label='LAP', color='#3498db')
ax3.plot(wavelength_spectrum, I2959_absorption, linewidth=3,
        label='Irgacure 2959', color='#e74c3c')
ax3.plot(wavelength_spectrum, EosinY_absorption, linewidth=3,
        label='Eosin Y', color='#2ecc71')

# Mark common LED wavelengths
for wl in wavelengths:
    ax3.axvline(wl, color='gray', linestyle=':', linewidth=2, alpha=0.5)
    ax3.text(wl, 1.05, f'{wl} nm', ha='center', fontsize=9)

ax3.set_xlabel('Wavelength (nm)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Normalized Absorption', fontsize=12, fontweight='bold')
ax3.set_title('Photoinitiator Absorption Spectra\n(Qualitative)',
             fontsize=13, fontweight='bold')
ax3.legend(loc='best', frameon=True, fancybox=True)
ax3.grid(True, alpha=0.3)
ax3.set_ylim(0, 1.2)

# Plot 4: Trade-off analysis
categories = ['Photodamage\n(Lower Better)', 'PI Efficiency\n(Higher Better)',
             'Equipment Cost\n(Lower Better)', 'Cell Safety\n(Higher Better)']

# Scores (0-10 scale, qualitative)
scores_365 = [4, 9, 7, 5]  # High damage, high efficiency, moderate cost, moderate safety
scores_385 = [6, 8, 8, 7]  # Moderate damage, good efficiency, moderate cost, good safety
scores_405 = [8, 6, 10, 9]  # Low damage, moderate efficiency, low cost, high safety

x = np.arange(len(categories))
width = 0.25

bars1 = ax4.bar(x - width, scores_365, width, label='365 nm',
               color=colors_wavelength[0], alpha=0.8, edgecolor='black')
bars2 = ax4.bar(x, scores_385, width, label='385 nm',
               color=colors_wavelength[1], alpha=0.8, edgecolor='black')
bars3 = ax4.bar(x + width, scores_405, width, label='405 nm',
               color=colors_wavelength[2], alpha=0.8, edgecolor='black')

ax4.set_ylabel('Score (0-10)', fontsize=12, fontweight='bold')
ax4.set_title('Wavelength Trade-off Analysis',
             fontsize=13, fontweight='bold')
ax4.set_xticks(x)
ax4.set_xticklabels(categories, fontsize=10)
ax4.legend(loc='upper right', frameon=True, fancybox=True)
ax4.grid(True, alpha=0.3, axis='y')
ax4.set_ylim(0, 10)

plt.tight_layout()
plt.show()

print("\n💡 Wavelength Selection Insights:")
print("   • 365 nm (UV-A):")
print("     ✓ High photoinitiator efficiency")
print("     ✓ Fast polymerization")
print("     ✗ Highest photodamage to cells")
print("     ✗ DNA damage risk")
print("\n   • 385 nm (Near-UV):")
print("     ✓ Good balance of efficiency and safety")
print("     ✓ Moderate cell damage")
print("     ~ Requires compatible photoinitiators")
print("\n   • 405 nm (Visible):")
print("     ✓ Lowest cell damage")
print("     ✓ Safest for sensitive cells")
print("     ✓ Cheap LED sources available")
print("     ✗ Lower photoinitiator efficiency (need higher dose)")
print("\n   • Recommendation: Use 405 nm unless speed is critical")

## Part 7: Summary and Key Takeaways

In [ ]:
print("="*80)
print("CHAPTER 4 EXERCISE 5: KEY LEARNING POINTS")
print("="*80)
print("""
1. BEER-LAMBERT LAW GOVERNS LIGHT PENETRATION
   → I(z) = I₀·exp(-αz)
   → Higher PI concentration → higher α → faster attenuation
   → Penetration depth Dₚ = 1/α (depth where intensity = I₀/e)
   → Typical Dₚ: 50-300 µm for bioprinting

2. WORKING CURVE PREDICTS CURE DEPTH
   → Cₐ = Dₚ·ln(E/Eₓ)
   → Logarithmic relationship: doubling exposure doesn't double cure depth
   → Critical energy Eₓ: minimum dose needed to initiate polymerization
   → Must calibrate for each resin formulation

3. EXPOSURE DOSE = INTENSITY × TIME
   → E = I·t (mJ/cm²)
   → Can trade intensity for time (but not always linearly)
   → Typical range: 20-200 mJ/cm² for bioprinting
   → Higher dose → deeper cure BUT more photodamage

4. PHOTOINITIATOR CONCENTRATION TRADE-OFFS
   → High PI (0.3-0.5%):
     • Fast polymerization
     • Thin layers only (low Dₚ)
     • Potential toxicity
   → Low PI (0.05-0.1%):
     • Thick layers possible (high Dₚ)
     • Slower polymerization
     • Better biocompatibility
   → Optimal: 0.1-0.2% for most applications

5. WAVELENGTH DETERMINES SAFETY
   → 365 nm: Fast but toxic (DNA damage)
   → 385 nm: Good balance
   → 405 nm: Safest but slower (recommended for cells)
   → Longer wavelength → less photodamage
   → Must match PI absorption spectrum

6. LAYER THICKNESS CONSTRAINTS
   → Typical: 50-100 µm per layer
   → Thinner layers:
     • Better Z-resolution
     • Slower printing
     • More interlayer bonding issues
   → Thicker layers:
     • Faster printing
     • Requires lower PI concentration
     • May compromise resolution

7. CELL VIABILITY CONSIDERATIONS
   → Photodamage increases with total exposure
   → Target: <500 mJ/cm² total for sensitive cells
   → Strategies to protect cells:
     • Use 405 nm instead of 365 nm
     • Minimize PI concentration
     • Reduce exposure time (higher intensity)
     • Add sacrificial absorbers

8. DLP vs SLA COMPARISON
   → DLP:
     • Entire layer at once (fast: 1-10 s/layer)
     • Pixel-limited resolution (25-100 µm)
     • Uniform exposure
   → SLA:
     • Point-by-point scanning (slower)
     • Better resolution (<25 µm)
     • More flexible intensity control

9. PRACTICAL DESIGN GUIDELINES
   → Wavelength: 405 nm (cell-friendly)
   → PI: LAP at 0.1-0.15% (good balance)
   → Intensity: 10-20 mW/cm² (practical)
   → Exposure: 5-15 s per layer
   → Layer thickness: 75 µm (standard)
   → Total dose: <200 mJ/cm² per layer

10. OPTIMIZATION STRATEGY
    1. Choose wavelength (405 nm recommended)
    2. Select target layer thickness (50-100 µm)
    3. Calculate required Dₚ from layer thickness
    4. Determine PI concentration from Dₚ and α
    5. Calculate exposure time from working curve
    6. Verify cell viability prediction
    7. Calibrate experimentally with test prints
""")
print("="*80)

## 🎓 REFLECTION QUESTIONS

### Question 1
**Derive the working curve equation Cₐ = Dₚ·ln(E/Eₓ) starting from the Beer-Lambert law. Explain the physical meaning of each term.**

### Question 2  
**You need to print 100 µm thick layers. Your resin has Eₓ = 25 mJ/cm² and you're using 15 mW/cm² intensity. Calculate the required PI concentration and exposure time. Assume ε = 100 L/(mol·cm).**

### Question 3
**Explain why doubling the photoinitiator concentration does NOT halve the required exposure time. Use the working curve equation in your explanation.**

### Question 4
**Compare the advantages and disadvantages of using 365 nm vs 405 nm light for printing hepatocyte-laden constructs (hepatocytes are UV-sensitive). What modifications would you make to the resin formulation for each wavelength?**

### Question 5
**Your prints show good X-Y resolution but poor Z-resolution (layers visible). The working curve parameters are: Dₚ = 200 µm, Eₓ = 20 mJ/cm², and you're using E = 60 mJ/cm². Calculate the cure depth and Z-resolution. How would you improve Z-resolution?**

## 📚 Additional Challenges (Optional)

### Challenge 1: Oxygen Inhibition
Oxygen inhibits free-radical polymerization. Model how O₂ concentration affects the critical energy Eₓ. What strategies can reduce oxygen inhibition?

### Challenge 2: Multi-Material Printing
Design a two-material system where material A (high PI) cures fast and material B (low PI) cures slow. How would you sequence the exposures?

### Challenge 3: Continuous DLP
Research continuous DLP (CDLP) with oxygen-permeable windows. Calculate the theoretical speed advantage over layer-by-layer printing.

### Challenge 4: Temperature Effects
Photopolymerization generates heat. Estimate the temperature rise in a 1 mm³ voxel during 10 s exposure at 20 mW/cm². Is this safe for cells?

## 🎯 Congratulations!

You've completed Exercise 5: Photopolymerization Kinetics and Light-Based Bioprinting!

**You now understand:**
- ✓ Beer-Lambert law and light penetration physics
- ✓ Working curve equation and cure depth prediction
- ✓ Photoinitiator concentration effects on penetration depth
- ✓ Wavelength-dependent cell photodamage
- ✓ Multi-parameter optimization for DLP/SLA
- ✓ Trade-offs between speed, resolution, and cell viability

**Next Steps:**
- Continue to Exercise 6: Scaffold Architecture and Porosity Design
- Review Chapter 4.4.1 for deeper understanding of light-based printing
- Explore research papers on photoclick chemistry and advanced photoinitiators

---

*Exercise created for Biofabrication Chapter 4 - Master's Level Bioengineering*